In [1]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Karan_Bais\\python\\Ml\\Projects\\Text_summary'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig():
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int
    gradient_checkpointing: bool
    optim: str

In [6]:
from Text_summary.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from Text_summary.utils.common import read_yaml, create_directories, print_gpu_utilization, print_summary

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(path_to_yaml=config_filepath)
        self.params = read_yaml(path_to_yaml=params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.Model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            #config
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            ##params
            num_epochs=self.params.num_epochs,
            warmup_steps=self.params.warmup_steps,
            per_device_train_batch_size=self.params.per_device_train_batch_size,
            per_device_eval_batch_size=self.params.per_device_eval_batch_size,
            weight_decay=self.params.weight_decay,
            logging_steps=self.params.logging_steps,
            evaluation_strategy=self.params.evaluation_strategy,
            eval_steps=self.params.eval_steps,
            save_steps=self.params.save_steps,
            gradient_accumulation_steps=self.params.gradient_accumulation_steps,
            gradient_checkpointing = self.params.gradient_checkpointing,
            optim=self.params.optim  
        )

        return model_trainer_config        

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

e:\Anaconda\envs\text-sum\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-22 12:16:27,225]: INFO: config: PyTorch version 2.3.0+cu118 available.]


In [11]:
print(torch.cuda.is_available())

True


In [12]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        sequence_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
        
        print_gpu_utilization()

        dataset = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
            gradient_checkpointing= self.config.gradient_checkpointing,
            optim=self.config.optim   # 'adamw' or 'adam'
        )

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
        #     per_device_train_batch_size=1, per_device_eval_batch_size=1,
        #     weight_decay=0.01, logging_steps=10,
        #     evaluation_strategy='steps', eval_steps=500, save_steps=100,
        #     gradient_accumulation_steps=16
        # ) 


        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=sequence_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            tokenizer=tokenizer
        )

        print_gpu_utilization()

        result = trainer.train()

        print_summary(result)

        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus_dailymail_cnn"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2520122824.py, line 27)

In [ ]:
try:
    config_manager = ConfigurationManager()
    model_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-07-19 11:34:43,057]: INFO: common: yaml file: config\config.yaml loaded]
[2024-07-19 11:34:43,061]: INFO: common: yaml file: params.yaml loaded]
[2024-07-19 11:34:43,063]: INFO: common: Created directory: artifacts]
[2024-07-19 11:34:43,065]: INFO: common: Created directory: artifacts/Model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory occupied: 2977 MB.
GPU memory occupied: 2977 MB.


e:\Anaconda\envs\text-sum\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


GPU memory occupied: 2977 MB.


In [ ]:
from pynvml import *

nvmlInit() 